# Aprendizaje No Supervisado: Clustering

<img src='https://files.realpython.com/media/centroids_iterations.247379590275.gif'>

El aprendizaje no supervisado es una técnica de aprendizaje automático que se emplea con el objeto de descubrir patrones en los datos. Por ejemplo, encontrar los "grupos" naturales de clientes (Segmentación de Clientes) en función de sus historiales de compra, buscando patrones, o correlaciones entre estos datos, permitiendo expresar dichos datos en forma comprimida. 

- La busqueda de patrones es conocida como Clustering o Agrupación
- La busqueda de correlación entre los datos es llamada Reducción de dimensionalidad.

**Datasets a utilizar:**

Un estudio realizado por PayScale Inc., proveedor on-line de datos de sueldos/salarios globales, encuestó a 1,2 millones de graduados, con un mínimo de 10 años de experiencia laboral. Las sujetos provenían de más de 300 escuelas de EE. UU., desde instituciones estatales hasta la Ivy League, y sus ingresos muestran que la materia en la que se especializa puede tener poco que ver con su poder adquisitivo a largo plazo. Se excluyeron a los encuestados que informaron tener títulos avanzados, incluidos M.B.A., M.D.s y J.D.s.


**Objetivo:**

Responder a la pregunta ¿Qué carreras universitarias tienden a tener mejores ingresos?

Veremos:
- Limpieza de datos y conversión de tipos de datos
- La diferencia de resultados entre escalar o no los datos.
- Determinación del número óptimo de clústeres.
- Modelado de clústeres con K-Medias.
- Visualización de resultados.

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.cluster import KMeans
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler

In [ ]:
data_DegreesPay = pd.read_csv('data/degrees-that-pay-back.csv')
data_DegreesPay.head()

In [ ]:
data_DegreesPay.info()

## Limpieza y curación

- Eliminación de caracteres especiales.
- Eliminación de "," de miles.
- Conversión de string a numeric

In [ ]:
# Limpieza de datos
data_DegreesPay = data_DegreesPay.astype(str).applymap(lambda x: x.replace(',', ''))\
                .applymap(lambda x: x.replace('$', ''))

data_DegreesPay.head()

In [ ]:
data_DegreesPay.dtypes

Se realizó una conversión de todas las columnas a tipo 'object', esto no es lo más correcto, para evitar hacer el código más extenso y complejo se hizo de esta manera.

Ahora debemos hacer la conversión a tipo numérico de las columnas que correspondan. 

El algoritmo estándar de k-medias no se aplica a los datos categóricos:
- El espacio muestral para datos categóricos es discreto y no tiene un origen natural.
- La función de distancia euclidiana en tal espacio no es realmente significativa.

In [ ]:
# Veamos sí alguna carrera se repite
print('Número de registros:', data_DegreesPay.shape[0])
print('Número de carreras:', data_DegreesPay['Undergraduate Major'].nunique())

## Conversión de tipo de datos

In [ ]:
# Guardemos las carreras en una variable, para usar luego
carreras = data_DegreesPay.iloc[:, 0]

# Eliminamos la columna categórica
df_DegreesPay = data_DegreesPay.iloc[:, 1:].copy()

# Conversión de cada columna a tipo númerico
for col in df_DegreesPay.columns:
    df_DegreesPay[col] = pd.to_numeric(df_DegreesPay[col], errors='coerce')

print(df_DegreesPay.info())
df_DegreesPay.head()

In [ ]:
# Expresemos el salario inicial en miles de dólares
df_DegreesPay['Starting Median Salary'] = df_DegreesPay['Starting Median Salary']/1000

## Modelado de Datos

Antes de modelar los datos con K-Means, apartemos algunos datos de prueba.

Y comencemos probando con 5 clusters:

In [ ]:
sample = df_DegreesPay.sample(n=5, random_state=28)
sample

In [ ]:
# Apartar datos de prueba
sample = df_DegreesPay.sample(n=5, random_state=28)

# Datos para modelar
modeling_sample = df_DegreesPay.drop(index=sample.index)

# Creamos la instancia del modelo para 5 números de clusters
k=5
model = KMeans(n_clusters=k, random_state=28)

# Ajustamos el modelo a los datos
model.fit(modeling_sample)

# Determinar las etiquetas de los clusters de la data de prueba
labels = model.predict(modeling_sample)
new_labels = model.predict(sample)

# Veamos las etiquetas de las carreras
print(*labels)

In [ ]:
# Veamos las etiquetas de los nuevas carreras
print(*new_labels)

In [ ]:
def plot_clusters(modeling_sample, sample, model, xy, axs, scaling=False):
    
    columns = modeling_sample.columns
    # Ajustar el pipeline a los datos
    model.fit(modeling_sample)

    # Obtención de etiquetas de clusters para los datos provistos
    labels = model.predict(modeling_sample)
    newlabels = model.predict(sample)


    if scaling:
        # Obteniendo los centroides de los clusters (en array)
        centroids = model[1].cluster_centers_
        centroids = scaler.inverse_transform(centroids)        
    else:
        # Obteniendo los centroides de los clusters (en array)
        centroids = model.cluster_centers_
        
    
    for par, ax in zip(xy, axs.ravel()):
        col1, col2 = par
        col1_name = modeling_sample.columns[col1]
        col2_name = modeling_sample.columns[col2]

        # Valores de columnas para graficar
        x1 = modeling_sample.iloc[:, col1]
        x2 = modeling_sample.iloc[:, col2]

        # Valores de los centroides para cada columna
        centroids_x1 = centroids[:, col1]
        centroids_x2 = centroids[:, col2]

        # Asignar colores a los clusters
        colores=['blue','red','green','cyan','orange','pink','purple', 'yellow']
        asignar_labels=[colores[label] for label in labels]
        asignar_newlabels=[colores[label] for label in newlabels]
        asignar_centroid=[colores[centroid] for centroid in range(centroids.shape[0])]

        # Graficar los puntos de datos según los valores de las columnas seleccionadas, c/u de un color según su cluster
        ax.scatter(x1, x2, c=asignar_labels, alpha=0.5)

        # Graficar los centroides según los valores de las columnas seleccionadas, c/u de un color según su cluster
        ax.scatter(centroids_x1, centroids_x2, c=asignar_centroid, marker='o', edgecolors='black', s=60)

        # Graficar los puntos nuevos según los valores de las columnas seleccionadas, c/u de un color según su cluster
        ax.scatter(sample.iloc[:, col1], sample.iloc[:, col2], c=asignar_newlabels, marker='+', s=90)
        ax.set_xlabel(col1_name, fontsize=12)
        ax.set_ylabel(col2_name, fontsize=12)
    plt.show()
    
    return labels

In [ ]:
xy = [(0, 1), (0, 2), (1, 2), (0, 4)]
fig, axs = plt.subplots(2, 2, figsize=(12, 12))

labels = plot_clusters(modeling_sample, sample, model, xy, axs)

## Escalado de datos

Pero dependiendo del tipo de datos con los que esté trabajando, es posible que la agrupación en clústeres no siempre sea tan buena. ¿Hay algo que pueda hacer en tales situaciones para mejorar su agrupación?

**Escalado estándar o Estandarización**

En la agrupación de KMeans, la varianza de una característica corresponde a su influencia en el algoritmo de agrupación. Para darle una oportunidad a cada característica, los datos deben transformarse para que las características tengan la misma variación. Esto se puede lograr con StandardScaler de scikit-learn. Esto transforma cada característica para que tenga media 0 y varianza 1. Las características "estandarizadas" pueden ser muy útiles en información.


In [ ]:
# Instanciar el método de escalado
scaler = StandardScaler()

# Crear instancia de K-Means
k=5
kmeans = KMeans(n_clusters=k, random_state=28)

# Create pipeline: pipeline
pipeline = make_pipeline(scaler, kmeans)

# Ajustar el pipeline a los datos
pipeline.fit(modeling_sample)

In [ ]:
xy = [(0, 1), (0, 2), (1, 2), (0, 4)]
fig, axs = plt.subplots(2, 2, figsize=(12, 12))

labels_scaling = plot_clusters(modeling_sample, sample, pipeline, xy, axs, scaling=True)

## Evaluación de la calidad de clusters

¿Cómo puede estar seguro de que 5 grupos es la elección correcta? ¿Cómo se puede evaluar la calidad de un agrupamiento?

- **Medición de la calidad de la agrupación**

Necesitamos una forma de medir la calidad de una agrupación que utilice solo las agrupaciones y las muestras en sí. Un buen agrupamiento tiene grupos reducidos, lo que significa que las muestras de cada grupo se agrupan, no se dispersan.

- **La inercia mide la calidad de la agrupación**

La "inercia" puede medir la dispersión de las muestras dentro de cada grupo. La inercia mide qué tan lejos están las muestras de sus centroides. 

Lo ideal es obtener grupos que no estén dispersos, por lo que los valores más bajos de inercia son mejores. 

K-Means tiene como objetivo colocar los clústeres de una manera que minimice la inercia.

- **Número de clusters óptimo**

Aquí hay una gráfica de los valores de inercia de agrupaciones del conjunto de datos del iris con diferentes números de agrupaciones. Nuestro modelo de kmeans con 3 grupos tiene una inercia relativamente baja, lo cual es genial. Pero observe que la inercia continúa disminuyendo lentamente. Entonces, ¿cuál es la mejor cantidad de clústeres para elegir?

- **¿Cuántos clusters elegir?**

Una buena agrupación tiene agrupaciones estrechas (baja inercia). Pero también, el menor número de clústeres. 

Una buena regla general, basada en la "Elbow Curve", consiste en elegir un codo en la gráfica de inercia, aquel punto donde la inercia comienza a disminuir más lentamente.


**Método del Codo o Elbow Method**
Este método gráfica el porcentaje de varianza vs el número de clusters. El codo de la curva indica el punto óptimo en el que agregar más grupos ya no explicará una cantidad significativa de la varianza. 

Los métodos más conocidos y empleados para obtener el número óptimo de clusters o agrupaciones, es el Método del Codo y el método de Siluetas o [Silhouette Method](https://scikit-learn.org/stable/auto_examples/cluster/plot_kmeans_silhouette_analysis.html?highlight=kmeans).


Generalmente no sabemos el número de clusters o agrupaciones son las óptimas, así que probemos con distintos números. Intentemos para k números de clusters desde 1 a 10:

In [ ]:
# k clusters de 1 a 10
ks = range(1, 11)
inertias = []

sample_scaled = scaler.fit_transform(modeling_sample)

# Creamos las diferentes instancias del modelo con cada k números de clusters
kmeans = [KMeans(n_clusters=k, random_state=28) for k in ks]

# Calculo de métrica para cada modelo
inertias = [model.fit(sample_scaled).inertia_ for model in kmeans]
    
# Graficar la variación explicada en función del número de clusters
plt.figure(figsize=(8,4))
plt.plot(ks, inertias, '-o')
plt.title('Elbow Curve')
plt.xlabel('Número de Clusters')
plt.ylabel('Inercia')
plt.xticks(ks)
plt.box(False)
plt.grid()
plt.show()

In [ ]:
# Create KMeans instance: kmeans
k_optimo = 3
kmeans = KMeans(n_clusters=k_optimo)

# Create pipeline: pipeline
pipeline = make_pipeline(scaler, kmeans)

# Ajustar el pipeline a los datos
pipeline.fit(modeling_sample)

In [ ]:
xy = [(0, 1), (0, 2), (1, 2), (0, 4)]
fig, axs = plt.subplots(2, 2, figsize=(12, 12))

clusters_labels = plot_clusters(modeling_sample, sample, pipeline, xy, axs, scaling=True)

In [ ]:
# Veamos las etiquetas de los clusters 
print(*clusters_labels)

In [ ]:
# Crear un dataframe con los resultados
results = modeling_sample.copy()
results['cluster'] = clusters_labels
results['carreras'] = carreras[results.index]

# Veamos las carreras para cada cluster
for cluster in range(k_optimo):
    carreras_cluster = results.loc[results['cluster']==cluster, 'carreras']
    print(f"Carreras del cluster {cluster}:\n{carreras_cluster.values}\n")
    

In [ ]:
results.head(3)